### Check GPU availability

In [1]:
!nvidia-smi

Sat Feb  9 17:44:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.145                Driver Version: 384.145                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   26C    P8    17W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 24%   36C    P8    17W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### Import libraries

In [2]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import copy
import numpy as np
import time
import json
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from pprint import pprint
from tqdm import tqdm_notebook
from idst_util import trivial
from idst_util import dstc2
from dstc2.dstc2_scripts import score

from plotly.graph_objs import Scatter, Layout, Histogram
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)

[nltk_data] Downloading package punkt to /home/is/andrei-
[nltk_data]     cc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Check DSTC2 availability and retrieve data

In [3]:
trivial.print_idst()

dstc2.check()

raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets(train_data_augmentation = True)

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!
INFO:root:+-

INFO:root:Extracting raw dev features


INFO:root:Reading dstc2_test.flist
INFO:root:Asserted 1117 dialogs for dstc2_test.flist
INFO:root:Extracting raw test features


INFO:root:Done!


### Set device

In [4]:
logging.info("+--------------------------------+")
logging.info("|             Device             |")
logging.info("+--------------------------------+")

GPU_ID = 1
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
if str(DEVICE) == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU {}".format(GPU_ID))

INFO:root:+--------------------------------+
INFO:root:|             Device             |
INFO:root:+--------------------------------+
INFO:root:Running on GPU 1


### Create vocabularies

In [5]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}

for raw_train_dialog in tqdm_notebook(raw_X_train):
    for raw_train_turn in raw_train_dialog["turns"]:
        tokens_scores = raw_train_turn["system"] + raw_train_turn["user"]
        for token_score in tokens_scores:
            token = token_score[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

logging.info("Done!")

INFO:root:+--------------------------------+
INFO:root:|          Vocabulary            |
INFO:root:+--------------------------------+
INFO:root:Creating token_to_index, index_to_token and token_to_count dictionaries


INFO:root:Done!


### Execution configuration

In [6]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)

# NOTE: we add +2 because of null and dontcare cases
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2 
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2

METHOD_DIM = len(ontology["method"])

REQUESTED_DIM = len(ontology["requestable"])

EMBEDDING_DIM = 170
ALTERED_EMBEDDING_DIM = 300
HIDDEN_DIM = 100

NUM_EPOCHS = 50
BATCH_SIZE = 10
PATIENCE = 4

PREDICTOR_DIM = 1

GOAL_LOSS_FUNCTION = nn.CrossEntropyLoss()
METHOD_LOSS_FUNCTION = nn.CrossEntropyLoss()
REQUESTED_LOSS_FUNCTION = nn.BCELoss()
PREDICTOR_LOSS_FUNCTION = nn.BCELoss()

logging.info("VOCABULARY_SIZE:\t\t\t{}".format(VOCABULARY_SIZE))

logging.info("GOAL_FOOD_DIM:\t\t\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t\t\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t\t\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t\t\t{}".format(GOAL_AREA_DIM))

logging.info("METHOD_DIM:\t\t\t\t{}".format(METHOD_DIM))

logging.info("REQUESTED_DIM:\t\t\t{}".format(REQUESTED_DIM))

logging.info("EMBEDDING_DIM:\t\t\t{}".format(EMBEDDING_DIM))
logging.info("ALTERED_EMBEDDING_DIM:\t\t{}".format(ALTERED_EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t\t\t{}".format(HIDDEN_DIM))

logging.info("PREDICTOR_DIM:\t\t\t{}".format(PREDICTOR_DIM))

logging.info("NUM_EPOCHS:\t\t\t\t{}".format(NUM_EPOCHS))
logging.info("BATCH_SIZE:\t\t\t\t{}".format(BATCH_SIZE))
logging.info("PATIENCE:\t\t\t\t{}".format(PATIENCE))

logging.info("GOAL_LOSS_FUNCTION:\t\t\t{}".format(GOAL_LOSS_FUNCTION))
logging.info("METHOD_LOSS_FUNCTION:\t\t\t{}".format(METHOD_LOSS_FUNCTION))
logging.info("REQUESTED_LOSS_FUNCTION:\t\t{}".format(REQUESTED_LOSS_FUNCTION))
logging.info("PREDICTOR_LOSS_FUNCTION:\t\t{}".format(PREDICTOR_LOSS_FUNCTION))

INFO:root:+--------------------------------+
INFO:root:|         Configuration          |
INFO:root:+--------------------------------+
INFO:root:VOCABULARY_SIZE:			1129
INFO:root:GOAL_FOOD_DIM:			93
INFO:root:GOAL_PRICERANGE_DIM:			5
INFO:root:GOAL_NAME_DIM:			115
INFO:root:GOAL_AREA_DIM:			7
INFO:root:METHOD_DIM:				5
INFO:root:REQUESTED_DIM:			8
INFO:root:EMBEDDING_DIM:			170
INFO:root:ALTERED_EMBEDDING_DIM:		300
INFO:root:HIDDEN_DIM:				100
INFO:root:PREDICTOR_DIM:			1
INFO:root:NUM_EPOCHS:				50
INFO:root:BATCH_SIZE:				10
INFO:root:PATIENCE:				4
INFO:root:GOAL_LOSS_FUNCTION:			CrossEntropyLoss()
INFO:root:METHOD_LOSS_FUNCTION:			CrossEntropyLoss()
INFO:root:REQUESTED_LOSS_FUNCTION:		BCELoss()
INFO:root:PREDICTOR_LOSS_FUNCTION:		BCELoss()


### Utilities

In [7]:
def get_index_and_score(turn, token_to_index, mode, device):
    
    indices = []
    scores = []
    
    if mode == "train": # LecTrack 4.3: Out-of-Vocabulary Words
        for system_token, system_token_score in turn["system"]:
            indices.append(token_to_index[system_token])
            scores.append(system_token_score)
        for user_token, user_token_score in turn["user"]:
            if np.random.binomial(n = 1, p = 0.1) == 1:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[user_token])
            scores.append(user_token_score)
    else:
        tokens_scores = turn["system"] + turn["user"]
        for token, score in tokens_scores:
            if token not in token_to_index:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[token])
            scores.append(score)
            
    assert len(indices) == len(scores)
    
    return torch.tensor(indices, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

# --------------------

class EarlyStopping():
    
    def __init__(self, min_delta = 0, patience = 0):
        
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
        self.stop_training = False
    
    def on_epoch_end(self, epoch, current_value):
        if np.greater((current_value - self.min_delta), self.best):
            self.best = current_value
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training

# --------------------
    
def get_incremental_index_and_percentage(percentage, length):
    
    incremental_index = int(np.around(percentage * length)) - 1
    new_percentage = np.around(((incremental_index + 1) / length), decimals = 2)
    
    return incremental_index, new_percentage

# --------------------

def make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset, percentage = 1.0):
    
    model_GoalPricerange = model_GoalPricerange.eval()
    model_GoalArea = model_GoalArea.eval()
    model_GoalName = model_GoalName.eval()
    model_GoalFood = model_GoalFood.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    percentage_points = []
    
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        
        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            model_GoalArea.hidden = model_GoalArea.init_hidden()
            model_GoalName.hidden = model_GoalName.init_hidden()
            model_GoalFood.hidden = model_GoalFood.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for turn_num, (raw_X_turn, raw_Y_turn) in enumerate(zip(raw_X_dialog["turns"], raw_Y_dialog["turns"])):

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)
                
                incremental_index, new_percentage_point = get_incremental_index_and_percentage(percentage = percentage, length = len(indices))
                percentage_points.append(new_percentage_point)
                
                goal_foods = model_GoalFood(indices, scores)
                goal_food = goal_foods[incremental_index]
                
                goal_priceranges = model_GoalPricerange(indices, scores)
                goal_pricerange = goal_priceranges[incremental_index]
                
                goal_names = model_GoalName(indices, scores)
                goal_name = goal_names[incremental_index]
                
                goal_areas = model_GoalArea(indices, scores)
                goal_area = goal_areas[incremental_index]
                
                requesteds = model_Requested(indices, scores)
                requested = requesteds[incremental_index]
                
                methods = model_Method(indices, scores)
                method = methods[incremental_index]
                
                turn = {}
                turn["num"] = turn_num
                turn["goal-labels"] = {}
                turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                turn["method-label"] = retrieve_output_Method(method, ontology)
                
                session["turns"].append(turn)
                
            tracker_json["sessions"].append(session)
            
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json, np.around(np.mean(np.array(percentage_points)), decimals = 2)

# --------------------
    
def get_scores(tracker, dataset, ontology):
    
    scores_dict = None

    if dataset == "dstc2_train":
        scores_dict = score.compute_score(dataset = "dstc2_train", dataroot = "dstc2/dstc2_traindev/data", tracker_output = tracker, ontology = ontology)
    elif dataset == "dstc2_dev":
        scores_dict = score.compute_score(dataset = "dstc2_dev", dataroot = "dstc2/dstc2_traindev/data", tracker_output = tracker, ontology = ontology)
    else: # dataset == "dstc2_test"
        scores_dict = score.compute_score(dataset = "dstc2_test", dataroot = "dstc2/dstc2_test/data", tracker_output = tracker, ontology = ontology)
            
    return scores_dict

# --------------------

def retrieve_gold_GoalPricerange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_output_GoalPricerange(output_tensor, ontology):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_pricerange_dict = {}
    goal_pricerange_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):     
        goal_pricerange_dict[ontology_informable_pricerange[index]] = output_tensor[index + 2].item()
    return goal_pricerange_dict

# --------------------

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

def retrieve_output_GoalArea(output_tensor, ontology):
    ontology_informable_area = ontology["informable"]["area"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_area_dict = {}
    goal_area_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_area_dict[ontology_informable_area[index]] = output_tensor[index + 2].item()
    return goal_area_dict

# --------------------

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_output_GoalName(output_tensor, ontology):
    ontology_informable_name = ontology["informable"]["name"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_name_dict = {}
    goal_name_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_name_dict[ontology_informable_name[index]] = output_tensor[index + 2].item()
    return goal_name_dict

# --------------------

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_output_GoalFood(output_tensor, ontology):
    ontology_informable_food = ontology["informable"]["food"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_food_dict = {}
    goal_food_dict["dontcare"] = output_tensor[1].item() 
    for index in range(len(output_tensor) - 2):
        goal_food_dict[ontology_informable_food[index]] = output_tensor[index + 2].item()
    return goal_food_dict

# --------------------

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    gold_requested = np.zeros(len(ontology_requestable), dtype = float)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            gold_requested[ontology_requestable.index(requested)] = 1.0
    return torch.tensor([gold_requested], dtype = torch.float, device = device)

def retrieve_output_Requested(output_tensor, ontology):
    ontology_requestable = ontology["requestable"]
    output_tensor = output_tensor.view(-1)
    requested_dict = {}
    for index in range(len(output_tensor)):
        probability_value = output_tensor[index].item()
        requested_dict[ontology_requestable[index]] = probability_value
    return requested_dict

# --------------------

def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_method = ontology["method"]
    raw_gold_method = raw_Y["method"]
    gold_method = ontology_method.index(raw_gold_method)
    return torch.tensor([gold_method], dtype = torch.long, device = device)

def retrieve_output_Method(output_tensor, ontology):
    ontology_method = ontology["method"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    method_dict = {}
    for index in range(len(output_tensor)):
        method_dict[ontology_method[index]] = output_tensor[index].item()
    return method_dict

### iDST Model

In [8]:
class iDSTModel(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_dim, altered_embedding_dim, hidden_dim, output_dim, device, is_requested = False):
        super(iDSTModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.is_requested = is_requested
        self.device = device
        self.output_dim = output_dim
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim)
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1), out_features = altered_embedding_dim) # +1 for the ASR-score
        self.lstm = nn.LSTM(input_size = altered_embedding_dim, hidden_size = hidden_dim)
        self.classifier = nn.Linear(in_features = hidden_dim, out_features = output_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))

    def forward(self, indices, scores):
        embeddings = self.embeddings(indices)
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        if self.is_requested:
            output_tensor = torch.sigmoid(self.classifier(lstm_out).view(-1, self.output_dim))
        else:
            output_tensor = F.log_softmax(self.classifier(lstm_out).view(-1, self.output_dim), dim = 1)
        return output_tensor

### Goal Pricerange Model

In [9]:
model_GoalPricerange = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 output_dim = GOAL_PRICERANGE_DIM,
                                 device = DEVICE).to(DEVICE)
optimizer_GoalPricerange = optim.Adam(model_GoalPricerange.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Area Model

In [10]:
model_GoalArea = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_AREA_DIM,
                           device = DEVICE).to(DEVICE)
optimizer_GoalArea = optim.Adam(model_GoalArea.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Name Model

In [11]:
model_GoalName = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_NAME_DIM,
                           device = DEVICE).to(DEVICE)
optimizer_GoalName = optim.Adam(model_GoalName.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Food Model

In [12]:
model_GoalFood = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_FOOD_DIM,
                           device = DEVICE).to(DEVICE)
optimizer_GoalFood = optim.Adam(model_GoalFood.parameters(), lr = 1e-3, amsgrad = True) 

### Requested Model

In [13]:
model_Requested = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                            embedding_dim = EMBEDDING_DIM,
                            altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                            hidden_dim = HIDDEN_DIM,
                            output_dim = REQUESTED_DIM,
                            device = DEVICE,
                            is_requested = True).to(DEVICE)
optimizer_Requested = optim.Adam(model_Requested.parameters(), lr = 1e-3, amsgrad = True) 

### Method Model

In [14]:
model_Method = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                         embedding_dim = EMBEDDING_DIM,
                         altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                         hidden_dim = HIDDEN_DIM,
                         output_dim = METHOD_DIM,
                         device = DEVICE).to(DEVICE)
optimizer_Method = optim.Adam(model_Method.parameters(), lr = 1e-3, amsgrad = True)

### Train iDST

In [15]:
goal_pricerange_early_stopping = EarlyStopping(patience = PATIENCE)
goal_area_early_stopping = EarlyStopping(patience = PATIENCE)
goal_name_early_stopping = EarlyStopping(patience = PATIENCE)
goal_food_early_stopping = EarlyStopping(patience = PATIENCE)
requested_early_stopping = EarlyStopping(patience = PATIENCE)
method_early_stopping = EarlyStopping(patience = PATIENCE)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    if not goal_pricerange_early_stopping.stop_training:
        model_GoalPricerange = model_GoalPricerange.train()
    if not goal_area_early_stopping.stop_training:
        model_GoalArea = model_GoalArea.train()
    if not goal_name_early_stopping.stop_training:
        model_GoalName = model_GoalName.train()
    if not goal_food_early_stopping.stop_training:
        model_GoalFood = model_GoalFood.train()
    if not requested_early_stopping.stop_training:
        model_Requested = model_Requested.train()
    if not method_early_stopping.stop_training:
        model_Method = model_Method.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        if not goal_pricerange_early_stopping.stop_training:
            optimizer_GoalPricerange.zero_grad()
            goal_pricerange_accumulated_loss = 0
        if not goal_area_early_stopping.stop_training:
            optimizer_GoalArea.zero_grad()
            goal_area_accumulated_loss = 0
        if not goal_name_early_stopping.stop_training:
            optimizer_GoalName.zero_grad()
            goal_name_accumulated_loss = 0
        if not goal_food_early_stopping.stop_training:
            optimizer_GoalFood.zero_grad()
            goal_food_accumulated_loss = 0
        if not requested_early_stopping.stop_training:
            optimizer_Requested.zero_grad()
            requested_accumulated_loss = 0
        if not method_early_stopping.stop_training:
            optimizer_Method.zero_grad()
            method_accumulated_loss = 0
        
        for raw_X_train_dialog, raw_Y_train_dialog in zip(raw_X_train[train_indices], raw_Y_train[train_indices]):
            
            if not goal_pricerange_early_stopping.stop_training:
                model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            if not goal_area_early_stopping.stop_training:
                model_GoalArea.hidden = model_GoalArea.init_hidden()
            if not goal_name_early_stopping.stop_training:
                model_GoalName.hidden = model_GoalName.init_hidden()
            if not goal_food_early_stopping.stop_training:
                model_GoalFood.hidden = model_GoalFood.init_hidden()
            if not requested_early_stopping.stop_training:
                model_Requested.hidden = model_Requested.init_hidden()
            if not method_early_stopping.stop_training:
                model_Method.hidden = model_Method.init_hidden()
                
            for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
                
                if not goal_pricerange_early_stopping.stop_training:
                    goal_pricerange_outputs = model_GoalPricerange(indices, scores)
                    gold_goal_pricerange = retrieve_gold_GoalPricerange(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(goal_pricerange_outputs))
                    goal_pricerange_accumulated_loss += GOAL_LOSS_FUNCTION(goal_pricerange_outputs, gold_goal_pricerange)
                if not goal_area_early_stopping.stop_training:
                    goal_area_outputs = model_GoalArea(indices, scores)
                    gold_goal_area = retrieve_gold_GoalArea(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(goal_area_outputs))
                    goal_area_accumulated_loss += GOAL_LOSS_FUNCTION(goal_area_outputs, gold_goal_area)
                if not goal_name_early_stopping.stop_training:
                    goal_name_outputs = model_GoalName(indices, scores)
                    gold_goal_name = retrieve_gold_GoalName(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(goal_name_outputs))
                    goal_name_accumulated_loss += GOAL_LOSS_FUNCTION(goal_name_outputs, gold_goal_name)
                if not goal_food_early_stopping.stop_training:
                    goal_food_outputs = model_GoalFood(indices, scores)
                    gold_goal_food = retrieve_gold_GoalFood(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(goal_food_outputs))
                    goal_food_accumulated_loss += GOAL_LOSS_FUNCTION(goal_food_outputs, gold_goal_food)
                if not requested_early_stopping.stop_training:
                    requested_outputs = model_Requested(indices, scores)
                    gold_requested = retrieve_gold_Requested(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(requested_outputs.size(0), 1)
                    requested_accumulated_loss += REQUESTED_LOSS_FUNCTION(requested_outputs, gold_requested)
                if not method_early_stopping.stop_training:
                    method_outputs = model_Method(indices, scores)
                    gold_method = retrieve_gold_Method(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(method_outputs))
                    method_accumulated_loss += METHOD_LOSS_FUNCTION(method_outputs, gold_method)
        
        if not goal_pricerange_early_stopping.stop_training:
            goal_pricerange_accumulated_loss.backward()
            optimizer_GoalPricerange.step()
        if not goal_area_early_stopping.stop_training:
            goal_area_accumulated_loss.backward()
            optimizer_GoalArea.step()
        if not goal_name_early_stopping.stop_training:
            goal_name_accumulated_loss.backward()
            optimizer_GoalName.step()
        if not goal_food_early_stopping.stop_training:
            goal_food_accumulated_loss.backward()
            optimizer_GoalFood.step()
        if not requested_early_stopping.stop_training:
            requested_accumulated_loss.backward()
            optimizer_Requested.step()
        if not method_early_stopping.stop_training:
            method_accumulated_loss.backward()
            optimizer_Method.step()
        
    dev_tracker, _ = make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method,
                                  raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
    
    dev_scores_dict = get_scores(dev_tracker, dataset = "dstc2_dev", ontology = ontology)
    
    logging.info(dev_scores_dict)

    goal_pricerange_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_pricerange_accuracy"]))
    goal_area_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_area_accuracy"]))
    goal_name_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_name_accuracy"]))
    goal_food_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_food_accuracy"]))
    requested_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["requested_all_accuracy"]))
    method_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["method_accuracy"]))
    
    if goal_pricerange_early_stopping.wait == 0:
        torch.save(model_GoalPricerange.state_dict(), "model_PANFMR_incremental_GoalPricerange.pt")
    if goal_area_early_stopping.wait == 0:
        torch.save(model_GoalArea.state_dict(), "model_PANFMR_incremental_GoalArea.pt")
    if goal_name_early_stopping.wait == 0:
        torch.save(model_GoalName.state_dict(), "model_PANFMR_incremental_GoalName.pt")
    if goal_food_early_stopping.wait == 0:
        torch.save(model_GoalFood.state_dict(), "model_PANFMR_incremental_GoalFood.pt")
    if requested_early_stopping.wait == 0:
        torch.save(model_Requested.state_dict(), "model_PANFMR_incremental_Requested.pt")
    if method_early_stopping.wait == 0:
        torch.save(model_Method.state_dict(), "model_PANFMR_incremental_Method.pt")
        
    if goal_pricerange_early_stopping.stop_training:
        model_GoalPricerange.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerange.pt"))
    if goal_area_early_stopping.stop_training:
        model_GoalArea.load_state_dict(torch.load("model_PANFMR_incremental_GoalArea.pt"))
    if goal_name_early_stopping.stop_training:
        model_GoalName.load_state_dict(torch.load("model_PANFMR_incremental_GoalName.pt"))
    if goal_food_early_stopping.stop_training:
        model_GoalFood.load_state_dict(torch.load("model_PANFMR_incremental_GoalFood.pt"))
    if requested_early_stopping.stop_training:
        model_Requested.load_state_dict(torch.load("model_PANFMR_incremental_Requested.pt"))
    if method_early_stopping.stop_training:
        model_Method.load_state_dict(torch.load("model_PANFMR_incremental_Method.pt"))
        
    if goal_pricerange_early_stopping.stop_training and goal_area_early_stopping.stop_training and goal_name_early_stopping.stop_training and \
        goal_food_early_stopping.stop_training and requested_early_stopping.stop_training and method_early_stopping.stop_training:
        break

INFO:root:Epoch	1/50


INFO:root:{'goal_food_accuracy': 0.4903462, 'goal_food_l2': 0.7435633, 'goal_pricerange_accuracy': 0.8835261, 'goal_pricerange_l2': 0.1793678, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1865353, 'goal_area_accuracy': 0.8433663, 'goal_area_l2': 0.2574117, 'goal_joint_accuracy': 0.4376955, 'goal_joint_l2': 0.7655974, 'requested_all_accuracy': 0.8308691, 'requested_all_l2': 0.2622236, 'method_accuracy': 0.8807553, 'method_l2': 0.1964218}
INFO:root:Epoch	2/50


INFO:root:{'goal_food_accuracy': 0.6850866, 'goal_food_l2': 0.4699368, 'goal_pricerange_accuracy': 0.9158357, 'goal_pricerange_l2': 0.1365933, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1865175, 'goal_area_accuracy': 0.8562444, 'goal_area_l2': 0.2289541, 'goal_joint_accuracy': 0.5750782, 'goal_joint_l2': 0.6031068, 'requested_all_accuracy': 0.9019113, 'requested_all_l2': 0.1422426, 'method_accuracy': 0.880238, 'method_l2': 0.1978085}
INFO:root:Epoch	3/50


INFO:root:{'goal_food_accuracy': 0.7173768, 'goal_food_l2': 0.3947068, 'goal_pricerange_accuracy': 0.9154368, 'goal_pricerange_l2': 0.1348328, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1863541, 'goal_area_accuracy': 0.8655286, 'goal_area_l2': 0.20957, 'goal_joint_accuracy': 0.5980188, 'goal_joint_l2': 0.5532732, 'requested_all_accuracy': 0.9278759, 'requested_all_l2': 0.1149272, 'method_accuracy': 0.8947232, 'method_l2': 0.1767082}
INFO:root:Epoch	4/50


INFO:root:{'goal_food_accuracy': 0.7280293, 'goal_food_l2': 0.3825936, 'goal_pricerange_accuracy': 0.9098524, 'goal_pricerange_l2': 0.1497601, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1865568, 'goal_area_accuracy': 0.8760108, 'goal_area_l2': 0.2043109, 'goal_joint_accuracy': 0.6107925, 'goal_joint_l2': 0.546179, 'requested_all_accuracy': 0.9412189, 'requested_all_l2': 0.0975105, 'method_accuracy': 0.8913606, 'method_l2': 0.1785192}
INFO:root:Epoch	5/50


...

INFO:root:{'goal_food_accuracy': 0.7473369, 'goal_food_l2': 0.37639, 'goal_pricerange_accuracy': 0.9202234, 'goal_pricerange_l2': 0.1298724, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1865353, 'goal_area_accuracy': 0.8793052, 'goal_area_l2': 0.1960002, 'goal_joint_accuracy': 0.6392075, 'goal_joint_l2': 0.5219743, 'requested_all_accuracy': 0.9567256, 'requested_all_l2': 0.0724307, 'method_accuracy': 0.8947232, 'method_l2': 0.1767082}
INFO:root:Epoch	21/50


INFO:root:{'goal_food_accuracy': 0.7473369, 'goal_food_l2': 0.37639, 'goal_pricerange_accuracy': 0.9202234, 'goal_pricerange_l2': 0.1298724, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1865353, 'goal_area_accuracy': 0.8793052, 'goal_area_l2': 0.1960002, 'goal_joint_accuracy': 0.6392075, 'goal_joint_l2': 0.5219743, 'requested_all_accuracy': 0.95348, 'requested_all_l2': 0.0723954, 'method_accuracy': 0.8947232, 'method_l2': 0.1767082}
INFO:root:Epoch	22/50


INFO:root:{'goal_food_accuracy': 0.7473369, 'goal_food_l2': 0.37639, 'goal_pricerange_accuracy': 0.9202234, 'goal_pricerange_l2': 0.1298724, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1865353, 'goal_area_accuracy': 0.8793052, 'goal_area_l2': 0.1960002, 'goal_joint_accuracy': 0.6392075, 'goal_joint_l2': 0.5219743, 'requested_all_accuracy': 0.9523981, 'requested_all_l2': 0.0743454, 'method_accuracy': 0.8947232, 'method_l2': 0.1767082}
INFO:root:Epoch	23/50


INFO:root:{'goal_food_accuracy': 0.7473369, 'goal_food_l2': 0.37639, 'goal_pricerange_accuracy': 0.9202234, 'goal_pricerange_l2': 0.1298724, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1865353, 'goal_area_accuracy': 0.8793052, 'goal_area_l2': 0.1960002, 'goal_joint_accuracy': 0.6392075, 'goal_joint_l2': 0.5219743, 'requested_all_accuracy': 0.9567256, 'requested_all_l2': 0.0713149, 'method_accuracy': 0.8947232, 'method_l2': 0.1767082}
INFO:root:Epoch	24/50


INFO:root:{'goal_food_accuracy': 0.7473369, 'goal_food_l2': 0.37639, 'goal_pricerange_accuracy': 0.9202234, 'goal_pricerange_l2': 0.1298724, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1865353, 'goal_area_accuracy': 0.8793052, 'goal_area_l2': 0.1960002, 'goal_joint_accuracy': 0.6392075, 'goal_joint_l2': 0.5219743, 'requested_all_accuracy': 0.9563649, 'requested_all_l2': 0.0719168, 'method_accuracy': 0.8947232, 'method_l2': 0.1767082}


### Load Goal Pricerange Model

In [16]:
model_GoalPricerange = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 output_dim = GOAL_PRICERANGE_DIM,
                                 device = DEVICE).to(DEVICE)
model_GoalPricerange.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerange.pt"))
model_GoalPricerange.eval()

iDSTModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=5, bias=True)
)

### Load Goal Area Model

In [17]:
model_GoalArea = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_AREA_DIM,
                           device = DEVICE).to(DEVICE)
model_GoalArea.load_state_dict(torch.load("model_PANFMR_incremental_GoalArea.pt"))
model_GoalArea.eval()

iDSTModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=7, bias=True)
)

### Load Goal Name Model

In [18]:
model_GoalName = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_NAME_DIM,
                           device = DEVICE).to(DEVICE)
model_GoalName.load_state_dict(torch.load("model_PANFMR_incremental_GoalName.pt"))
model_GoalName.eval()

iDSTModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=115, bias=True)
)

### Load Goal Food Model

In [19]:
model_GoalFood = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_FOOD_DIM,
                           device = DEVICE).to(DEVICE)
model_GoalFood.load_state_dict(torch.load("model_PANFMR_incremental_GoalFood.pt"))
model_GoalFood.eval()

iDSTModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=93, bias=True)
)

### Load Requested Model

In [20]:
model_Requested = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                            embedding_dim = EMBEDDING_DIM,
                            altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                            hidden_dim = HIDDEN_DIM,
                            output_dim = REQUESTED_DIM,
                            device = DEVICE,
                            is_requested = True).to(DEVICE)
model_Requested.load_state_dict(torch.load("model_PANFMR_incremental_Requested.pt"))
model_Requested.eval()

iDSTModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=8, bias=True)
)

### Load Method Model

In [21]:
model_Method = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                         embedding_dim = EMBEDDING_DIM,
                         altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                         hidden_dim = HIDDEN_DIM,
                         output_dim = METHOD_DIM,
                         device = DEVICE).to(DEVICE)
model_Method.load_state_dict(torch.load("model_PANFMR_incremental_Method.pt"))
model_Method.eval()

iDSTModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=5, bias=True)
)

### Print scores

In [22]:
dev_tracker, _ = make_tracker(model_GoalPricerange,
                              model_GoalArea,
                              model_GoalName,
                              model_GoalFood,
                              model_Requested,
                              model_Method,
                              raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev", ontology = ontology)

{'goal_food_accuracy': 0.7473369,
 'goal_food_l2': 0.37639,
 'goal_pricerange_accuracy': 0.9202234,
 'goal_pricerange_l2': 0.1298724,
 'goal_name_accuracy': 0.9065421,
 'goal_name_l2': 0.1865353,
 'goal_area_accuracy': 0.8793052,
 'goal_area_l2': 0.1960002,
 'goal_joint_accuracy': 0.6392075,
 'goal_joint_l2': 0.5219743,
 'requested_all_accuracy': 0.9621349,
 'requested_all_l2': 0.0675933,
 'method_accuracy': 0.8947232,
 'method_l2': 0.1767082}

In [23]:
test_tracker, _ = make_tracker(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test", ontology = ontology)

{'goal_food_accuracy': 0.7867474,
 'goal_food_l2': 0.3400614,
 'goal_pricerange_accuracy': 0.9032499,
 'goal_pricerange_l2': 0.1570489,
 'goal_name_accuracy': 0.961326,
 'goal_name_l2': 0.0771895,
 'goal_area_accuracy': 0.8755821,
 'goal_area_l2': 0.2020645,
 'goal_joint_accuracy': 0.6488802,
 'goal_joint_l2': 0.5277477,
 'requested_all_accuracy': 0.9640563,
 'requested_all_l2': 0.056568,
 'method_accuracy': 0.9195402,
 'method_l2': 0.1317837}

### SUPERVISED TURN TAKING

### Predictor utilities

In [24]:
def get_gaps_vector(vector):
    gaps_vector = []
    
    for value in vector:
        if value == vector[-1]:
            gaps_vector.append(0)
        else:
            gaps_vector.append(1)
    
    return gaps_vector

#--------------------

def get_predictor_score(dataset, session_id, turn):
    start_time = time.time()
    
    tracker_json = {}
    tracker_json["dataset"] = dataset
    tracker_json["sessions"] = []
    
    session = {}
    session["session-id"] = session_id
    session["turns"] = [copy.deepcopy(turn)] # NOTE: copy here is needed since the scoring script is going to modify the dict
    
    tracker_json["sessions"].append(session)
        
    end_time = time.time()
    
    tracker_json["wall-time"] = end_time - start_time
    
    return get_scores(tracker_json, dataset, ontology)

#--------------------

def get_predictor_Y(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, dataset, ontology):
    
    predictor_Y = []
    
    if dataset == "dstc2_train":
        raw_X = raw_X_train
        raw_Y = raw_Y_train
    elif dataset == "dstc2_dev":
        raw_X = raw_X_dev
        raw_Y = raw_Y_dev
    else: # dataset == "dstc2_test"
        raw_X = raw_X_test
        raw_Y = raw_Y_test
    
    with torch.no_grad():
        
        model_GoalPricerange = model_GoalPricerange.eval()
        model_GoalArea = model_GoalArea.eval()
        model_GoalName = model_GoalName.eval()
        model_GoalFood = model_GoalFood.eval()
        model_Requested = model_Requested.eval()
        model_Method = model_Method.eval()

        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):

            assert(raw_X_dialog["session-id"] == raw_Y_dialog["session-id"])

            predictor_Y_dialog = {}
            predictor_Y_dialog["session-id"] = raw_Y_dialog["session-id"]
            predictor_Y_dialog["turns"] = []

            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            model_GoalArea.hidden = model_GoalArea.init_hidden()
            model_GoalName.hidden = model_GoalName.init_hidden()
            model_GoalFood.hidden = model_GoalFood.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            for turn_num, (raw_X_turn, raw_Y_turn) in enumerate(zip(raw_X_dialog["turns"], raw_Y_dialog["turns"])):
                
                predictor_Y_turn = {}
                predictor_Y_turn["goal_pricerange_predictor"] = []
                predictor_Y_turn["goal_area_predictor"] = []
                predictor_Y_turn["goal_name_predictor"] = []
                predictor_Y_turn["goal_food_predictor"] = []
                predictor_Y_turn["requested_predictor"] = []
                predictor_Y_turn["method_predictor"] = []
            
                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)

                goal_foods = model_GoalFood(indices, scores)
                goal_priceranges = model_GoalPricerange(indices, scores)
                goal_names = model_GoalName(indices, scores)
                goal_areas = model_GoalArea(indices, scores)
                requesteds = model_Requested(indices, scores)
                methods = model_Method(indices, scores)

                for goal_food, goal_pricerange, goal_name, goal_area, requested, method in zip(goal_foods, goal_priceranges, goal_names, goal_areas, requesteds, methods):
                    
                    turn = {}
                    turn["num"] = turn_num
                    turn["goal-labels"] = {}
                    turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                    turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                    turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                    turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                    turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                    turn["method-label"] = retrieve_output_Method(method, ontology)

                    scores_dict = get_predictor_score(dataset, raw_X_dialog["session-id"], turn)
                    
                    predictor_Y_turn["goal_pricerange_predictor"].append(scores_dict["goal_pricerange_accuracy"])
                    predictor_Y_turn["goal_area_predictor"].append(scores_dict["goal_area_accuracy"])
                    predictor_Y_turn["goal_name_predictor"].append(scores_dict["goal_name_accuracy"])
                    predictor_Y_turn["goal_food_predictor"].append(scores_dict["goal_food_accuracy"])
                    predictor_Y_turn["requested_predictor"].append(scores_dict["requested_all_accuracy"])
                    predictor_Y_turn["method_predictor"].append(scores_dict["method_accuracy"])
                
                predictor_Y_turn["goal_pricerange_predictor"] = get_gaps_vector(predictor_Y_turn["goal_pricerange_predictor"])
                predictor_Y_turn["goal_area_predictor"] = get_gaps_vector(predictor_Y_turn["goal_area_predictor"])
                predictor_Y_turn["goal_name_predictor"] = get_gaps_vector(predictor_Y_turn["goal_name_predictor"])
                predictor_Y_turn["goal_food_predictor"] = get_gaps_vector(predictor_Y_turn["goal_food_predictor"])
                predictor_Y_turn["requested_predictor"] = get_gaps_vector(predictor_Y_turn["requested_predictor"])
                predictor_Y_turn["method_predictor"] = get_gaps_vector(predictor_Y_turn["method_predictor"])
                
                predictor_Y_dialog["turns"].append(predictor_Y_turn)

            predictor_Y.append(predictor_Y_dialog)

        return np.array(predictor_Y)

#--------------------    
    
def get_predictor_loss(model_GoalPricerangePredictor, model_GoalAreaPredictor, model_GoalNamePredictor,
                       model_GoalFoodPredictor, model_RequestedPredictor, model_MethodPredictor,
                       raw_X, predictor_Y, device):

    model_GoalPricerangePredictor = model_GoalPricerangePredictor.eval()
    model_GoalAreaPredictor = model_GoalAreaPredictor.eval()
    model_GoalNamePredictor = model_GoalNamePredictor.eval()
    model_GoalFoodPredictor = model_GoalFoodPredictor.eval()
    model_RequestedPredictor = model_RequestedPredictor.eval()
    model_MethodPredictor = model_MethodPredictor.eval()
    
    losses_dict = {}
    losses_dict["goal_pricerange_loss"] = 0
    losses_dict["goal_area_loss"] = 0
    losses_dict["goal_name_loss"] = 0
    losses_dict["goal_food_loss"] = 0
    losses_dict["requested_all_loss"] = 0
    losses_dict["method_loss"] = 0
    
    with torch.no_grad():
                           
        for raw_X_dialog, predictor_Y_dialog in zip(raw_X, predictor_Y):

                model_GoalPricerangePredictor.hidden = model_GoalPricerangePredictor.init_hidden()
                model_GoalAreaPredictor.hidden = model_GoalAreaPredictor.init_hidden()
                model_GoalNamePredictor.hidden = model_GoalNamePredictor.init_hidden()
                model_GoalFoodPredictor.hidden = model_GoalFoodPredictor.init_hidden()
                model_RequestedPredictor.hidden = model_RequestedPredictor.init_hidden()
                model_MethodPredictor.hidden = model_MethodPredictor.init_hidden()

                for raw_X_turn, predictor_Y_turn in zip(raw_X_dialog["turns"], predictor_Y_dialog["turns"]):

                    indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)

                    goal_pricerange_outputs = model_GoalPricerangePredictor(indices, scores)
                    gold_goal_pricerange = torch.t(torch.tensor([predictor_Y_turn["goal_pricerange_predictor"]], dtype = torch.float, device = DEVICE))
                    losses_dict["goal_pricerange_loss"] += PREDICTOR_LOSS_FUNCTION(goal_pricerange_outputs, gold_goal_pricerange)
                    
                    goal_area_outputs = model_GoalAreaPredictor(indices, scores)
                    gold_goal_area = torch.t(torch.tensor([predictor_Y_turn["goal_area_predictor"]], dtype = torch.float, device = DEVICE))
                    losses_dict["goal_area_loss"] += PREDICTOR_LOSS_FUNCTION(goal_area_outputs, gold_goal_area)
                    
                    goal_name_outputs = model_GoalNamePredictor(indices, scores)
                    gold_goal_name = torch.t(torch.tensor([predictor_Y_turn["goal_name_predictor"]], dtype = torch.float, device = DEVICE))
                    losses_dict["goal_name_loss"] += PREDICTOR_LOSS_FUNCTION(goal_name_outputs, gold_goal_name)
                    
                    goal_food_outputs = model_GoalFoodPredictor(indices, scores)
                    gold_goal_food = torch.t(torch.tensor([predictor_Y_turn["goal_food_predictor"]], dtype = torch.float, device = DEVICE))
                    losses_dict["goal_food_loss"] += PREDICTOR_LOSS_FUNCTION(goal_food_outputs, gold_goal_food)
                    
                    requested_outputs = model_RequestedPredictor(indices, scores)
                    gold_requested = torch.t(torch.tensor([predictor_Y_turn["requested_predictor"]], dtype = torch.float, device = DEVICE))
                    losses_dict["requested_all_loss"] += PREDICTOR_LOSS_FUNCTION(requested_outputs, gold_requested)
                    
                    method_outputs = model_MethodPredictor(indices, scores)
                    gold_method = torch.t(torch.tensor([predictor_Y_turn["method_predictor"]], dtype = torch.float, device = DEVICE))
                    losses_dict["method_loss"] += PREDICTOR_LOSS_FUNCTION(method_outputs, gold_method)
    
    losses_dict["goal_pricerange_loss"] = losses_dict["goal_pricerange_loss"].item()
    losses_dict["goal_area_loss"] = losses_dict["goal_area_loss"].item()
    losses_dict["goal_name_loss"] = losses_dict["goal_name_loss"].item()
    losses_dict["goal_food_loss"] = losses_dict["goal_food_loss"].item()
    losses_dict["requested_all_loss"] = losses_dict["requested_all_loss"].item()
    losses_dict["method_loss"] = losses_dict["method_loss"].item()
    
    return losses_dict

#--------------------

def make_predictor_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method,
                           model_GoalPricerangePredictor, model_GoalAreaPredictor, model_GoalNamePredictor, model_GoalFoodPredictor, model_RequestedPredictor, model_MethodPredictor,
                           raw_X, raw_Y, dataset, threshold = 0.5):
    
    model_GoalPricerange = model_GoalPricerange.eval()
    model_GoalArea = model_GoalArea.eval()
    model_GoalName = model_GoalName.eval()
    model_GoalFood = model_GoalFood.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    percentage_points = []
    
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        
        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            model_GoalArea.hidden = model_GoalArea.init_hidden()
            model_GoalName.hidden = model_GoalName.init_hidden()
            model_GoalFood.hidden = model_GoalFood.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            model_GoalPricerangePredictor.hidden = model_GoalPricerangePredictor.init_hidden()
            model_GoalAreaPredictor.hidden = model_GoalAreaPredictor.init_hidden()
            model_GoalNamePredictor.hidden = model_GoalNamePredictor.init_hidden()
            model_GoalFoodPredictor.hidden = model_GoalFoodPredictor.init_hidden()
            model_RequestedPredictor.hidden = model_RequestedPredictor.init_hidden()
            model_MethodPredictor.hidden = model_MethodPredictor.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for turn_num, (raw_X_turn, raw_Y_turn) in enumerate(zip(raw_X_dialog["turns"], raw_Y_dialog["turns"])):
                
                take_turn = False
                
                turn = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)
                
                goal_priceranges = model_GoalPricerange(indices, scores)
                goal_names = model_GoalName(indices, scores)
                goal_areas = model_GoalArea(indices, scores)
                goal_foods = model_GoalFood(indices, scores)
                requesteds = model_Requested(indices, scores)
                methods = model_Method(indices, scores)
                
                goal_priceranges_predictor = model_GoalPricerangePredictor(indices, scores)
                goal_names_predictor = model_GoalNamePredictor(indices, scores)
                goal_areas_predictor = model_GoalAreaPredictor(indices, scores)
                goal_foods_predictor = model_GoalFoodPredictor(indices, scores)
                requesteds_predictor = model_RequestedPredictor(indices, scores)
                methods_predictor = model_MethodPredictor(indices, scores)
                
                for index, (goal_pricerange_predictor, goal_name_predictor, goal_area_predictor, goal_food_predictor, requested_predictor, method_predictor) in\
                    enumerate(zip(goal_priceranges_predictor, goal_names_predictor, goal_areas_predictor, goal_foods_predictor, requesteds_predictor, methods_predictor)):
                    
                    if (goal_pricerange_predictor.item() <= threshold) and (goal_name_predictor.item() <= threshold) and (goal_area_predictor.item() <= threshold) and\
                        (goal_food_predictor.item() <= threshold) and (requested_predictor.item() <= threshold) and (method_predictor.item() <= threshold):
                        
                        turn["num"] = turn_num
                        turn["goal-labels"] = {}
                        turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_foods[index], ontology)
                        turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_priceranges[index], ontology)
                        turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_names[index], ontology)
                        turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_areas[index], ontology)
                        turn["requested-slots"] = retrieve_output_Requested(requesteds[index], ontology)
                        turn["method-label"] = retrieve_output_Method(methods[index], ontology)
                        
                        session["turns"].append(turn)
                        percentage_point = np.around((index + 1) / (len(raw_X_turn["system"]) + len(raw_X_turn["user"])), decimals = 2)
                        percentage_points.append(percentage_point)
                        break
                        
                if not turn: # in case if no prediction for the turn
                    turn["num"] = turn_num
                    turn["goal-labels"] = {}
                    turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_foods[-1], ontology)
                    turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_priceranges[-1], ontology)
                    turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_names[-1], ontology)
                    turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_areas[-1], ontology)
                    turn["requested-slots"] = retrieve_output_Requested(requesteds[-1], ontology)
                    turn["method-label"] = retrieve_output_Method(methods[-1], ontology)
                    percentage_points.append(1.0)
                    session["turns"].append(turn)
                
            tracker_json["sessions"].append(session)
            
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json, np.around(np.mean(np.array(percentage_points)), decimals = 2), percentage_points

#--------------------
    
def frange(start, stop, step):
    i = start
    while i < stop:
        i = np.around(i, decimals = 2)
        yield i
        i += step    
    
#--------------------

def plotly_plot_histogram(data, dataset):
    if dataset == "dstc2_train":
        dataset = "TRAIN"
    elif dataset == "dstc2_dev":
        dataset = "DEV"
    else:
        dataset = "TEST"
    plotly.offline.iplot({"data": [Histogram(x = data, histnorm = "probability", xbins = dict(size = 0.025), marker = dict(color = "#3498db"))],
                          "layout": Layout(title = "<b>{} Percentage - Accuracy</b>".format(dataset),
                                           xaxis = dict(title = "<b>Percentage</b>",
                                                        dtick = 0.025,
                                                        titlefont = dict(color = "#34495e")),
                                           yaxis = dict(title = "<b>Normalized counts</b>",
                                                        dtick = 0.025,
                                                        titlefont = dict(color = "#34495e")),
                                           margin = Margin(b = 150))})

#--------------------

def plotly_plot_incremental(goal_pricerange_accuracies, goal_area_accuracies, goal_name_accuracies, goal_food_accuracies,
                            requested_accuracies, method_accuracies, percentages, predictor_percentage_point, predictor_goal_pricerange_accuracy,
                            predictor_goal_area_accuracy, predictor_goal_name_accuracy, predictor_goal_food_accuracy,
                            predictor_requested_accuracy, predictor_method_accuracy, dataset):
    
    if dataset == "dstc2_train":
        dataset = "TRAIN"
    elif dataset == "dstc2_dev":
        dataset = "DEV"
    else:
        dataset = "TEST"
        
    plotly.offline.iplot({"data": [Scatter(x = percentages, y = goal_pricerange_accuracies, mode = "lines+markers", name = "{} Goal Pricerange Accuracy".format(dataset), marker = dict(color = "#1abc9c")),
                                   Scatter(x = percentages, y = goal_area_accuracies, mode = "lines+markers", name = "{} Goal Area Accuracy".format(dataset), marker = dict(color = "#3498db")),
                                   Scatter(x = percentages, y = goal_name_accuracies, mode = "lines+markers", name = "{} Goal Name Accuracy".format(dataset), marker = dict(color = "#9b59b6")),
                                   Scatter(x = percentages, y = goal_food_accuracies, mode = "lines+markers", name = "{} Goal Food Accuracy".format(dataset), marker = dict(color = "#e74c3c")),
                                   Scatter(x = percentages, y = requested_accuracies, mode = "lines+markers", name = "{} Requested Accuracy".format(dataset), marker = dict(color = "#34495e")),
                                   Scatter(x = percentages, y = method_accuracies, mode = "lines+markers", name = "{} Method Accuracy".format(dataset), marker = dict(color = "#f1c40f")),
                                   Scatter(x = [predictor_percentage_point], y = [predictor_goal_pricerange_accuracy], mode = "markers", name = "{} (iSTT) Goal Pricerange Accuracy".format(dataset), marker = dict(color = "#1abc9c")),
                                   Scatter(x = [predictor_percentage_point], y = [predictor_goal_area_accuracy], mode = "markers", name = "{} (iSTT) Goal Area Accuracy".format(dataset), marker = dict(color = "#3498db")),
                                   Scatter(x = [predictor_percentage_point], y = [predictor_goal_name_accuracy], mode = "markers", name = "{} (iSTT) Goal Name Accuracy".format(dataset), marker = dict(color = "#9b59b6")),
                                   Scatter(x = [predictor_percentage_point], y = [predictor_goal_food_accuracy], mode = "markers", name = "{} (iSTT) Goal Food Accuracy".format(dataset), marker = dict(color = "#e74c3c")),
                                   Scatter(x = [predictor_percentage_point], y = [predictor_requested_accuracy], mode = "markers", name = "{} (iSTT) Requested Accuracy".format(dataset), marker = dict(color = "#34495e")),
                                   Scatter(x = [predictor_percentage_point], y = [predictor_method_accuracy], mode = "markers", name = "{} (iSTT) Method Accuracy".format(dataset), marker = dict(color = "#f1c40f"))],      
                            "layout": Layout(title = "<b>{} Percentage - Accuracy</b>".format(dataset),
                                             xaxis = dict(title = "<b>Percentage</b>",
                                                          dtick = 0.05,
                                                          titlefont = dict(color = "#34495e")),
                                             yaxis = dict(title = "<b>Accuracy</b>",
                                                          dtick = 0.025,
                                                          titlefont = dict(color = "#34495e")),
                                             margin = Margin(b = 150))})

### Compute and save development set predictor labels

In [25]:
dev_predictor_Y = get_predictor_Y(model_GoalPricerange,
                                  model_GoalArea,
                                  model_GoalName,
                                  model_GoalFood,
                                  model_Requested,
                                  model_Method,
                                  "dstc2_dev",
                                  ontology)
np.save("dev_predictor_Y", dev_predictor_Y)

### Load development set predictor labels

In [26]:
dev_predictor_Y = np.load("dev_predictor_Y.npy")

### Compute and save train set predictor labels

In [27]:
train_predictor_Y = get_predictor_Y(model_GoalPricerange,
                                    model_GoalArea,
                                    model_GoalName,
                                    model_GoalFood,
                                    model_Requested,
                                    model_Method,
                                    "dstc2_train",
                                    ontology)
np.save("train_predictor_Y", train_predictor_Y)

### Load train set predictor labels

In [28]:
train_predictor_Y = np.load("train_predictor_Y.npy")

### iDST Predictor Model

In [29]:
class iDSTPredictorModel(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_dim, altered_embedding_dim, hidden_dim, output_dim, device):
        super(iDSTPredictorModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.output_dim = output_dim
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim)
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1), out_features = altered_embedding_dim) # +1 for the ASR-score
        self.lstm = nn.LSTM(input_size = altered_embedding_dim, hidden_size = hidden_dim)
        self.classifier = nn.Linear(in_features = hidden_dim, out_features = output_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))

    def forward(self, indices, scores):
        embeddings = self.embeddings(indices)
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        output_tensor = torch.sigmoid(self.classifier(lstm_out).view(-1, self.output_dim))
        return output_tensor

### Goal Pricerange Predictor Model

In [30]:
model_GoalPricerangePredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                   embedding_dim = EMBEDDING_DIM,
                                                   altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                   hidden_dim = HIDDEN_DIM,
                                                   output_dim = PREDICTOR_DIM,
                                                   device = DEVICE).to(DEVICE)
optimizer_GoalPricerangePredictor = optim.Adam(model_GoalPricerangePredictor.parameters(), lr = 1e-3, amsgrad = True)

### Goal Area Predictor Model

In [31]:
model_GoalAreaPredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                             embedding_dim = EMBEDDING_DIM,
                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                             hidden_dim = HIDDEN_DIM,
                                             output_dim = PREDICTOR_DIM,
                                             device = DEVICE).to(DEVICE)
optimizer_GoalAreaPredictor = optim.Adam(model_GoalAreaPredictor.parameters(), lr = 1e-3, amsgrad = True)

### Goal Name Predictor Model

In [32]:
model_GoalNamePredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                             embedding_dim = EMBEDDING_DIM,
                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                             hidden_dim = HIDDEN_DIM,
                                             output_dim = PREDICTOR_DIM,
                                             device = DEVICE).to(DEVICE)
optimizer_GoalNamePredictor = optim.Adam(model_GoalNamePredictor.parameters(), lr = 1e-3, amsgrad = True)

### Goal Food Predictor Model

In [33]:
model_GoalFoodPredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                             embedding_dim = EMBEDDING_DIM,
                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                             hidden_dim = HIDDEN_DIM,
                                             output_dim = PREDICTOR_DIM,
                                             device = DEVICE).to(DEVICE)
optimizer_GoalFoodPredictor = optim.Adam(model_GoalFoodPredictor.parameters(), lr = 1e-3, amsgrad = True)

### Requested Predictor Model

In [34]:
model_RequestedPredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                             embedding_dim = EMBEDDING_DIM,
                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                             hidden_dim = HIDDEN_DIM,
                                             output_dim = PREDICTOR_DIM,
                                             device = DEVICE).to(DEVICE)
optimizer_RequestedPredictor = optim.Adam(model_RequestedPredictor.parameters(), lr = 1e-3, amsgrad = True)

### Method Predictor Model

In [35]:
model_MethodPredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                           embedding_dim = EMBEDDING_DIM,
                                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                           hidden_dim = HIDDEN_DIM,
                                           output_dim = PREDICTOR_DIM,
                                           device = DEVICE).to(DEVICE)
optimizer_MethodPredictor = optim.Adam(model_MethodPredictor.parameters(), lr = 1e-3, amsgrad = True)

### Train iDST Predictor

In [36]:
goal_pricerange_predictor_early_stopping = EarlyStopping(patience = PATIENCE)
goal_area_predictor_early_stopping = EarlyStopping(patience = PATIENCE)
goal_name_predictor_early_stopping = EarlyStopping(patience = PATIENCE)
goal_food_predictor_early_stopping = EarlyStopping(patience = PATIENCE)
requested_predictor_early_stopping = EarlyStopping(patience = PATIENCE)
method_predictor_early_stopping = EarlyStopping(patience = PATIENCE)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    if not goal_pricerange_predictor_early_stopping.stop_training:
        model_GoalPricerangePredictor = model_GoalPricerangePredictor.train()
    if not goal_area_predictor_early_stopping.stop_training:
        model_GoalAreaPredictor = model_GoalAreaPredictor.train()
    if not goal_name_predictor_early_stopping.stop_training:
        model_GoalNamePredictor = model_GoalNamePredictor.train()
    if not goal_food_predictor_early_stopping.stop_training:
        model_GoalFoodPredictor = model_GoalFoodPredictor.train()
    if not requested_predictor_early_stopping.stop_training:
        model_RequestedPredictor = model_RequestedPredictor.train()
    if not method_predictor_early_stopping.stop_training:
        model_MethodPredictor = model_MethodPredictor.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        if not goal_pricerange_predictor_early_stopping.stop_training:
            optimizer_GoalPricerangePredictor.zero_grad()
            goal_pricerange_accumulated_loss = 0
        if not goal_area_predictor_early_stopping.stop_training:
            optimizer_GoalAreaPredictor.zero_grad()
            goal_area_accumulated_loss = 0
        if not goal_name_predictor_early_stopping.stop_training:
            optimizer_GoalNamePredictor.zero_grad()
            goal_name_accumulated_loss = 0
        if not goal_food_predictor_early_stopping.stop_training:
            optimizer_GoalFoodPredictor.zero_grad()
            goal_food_accumulated_loss = 0
        if not requested_predictor_early_stopping.stop_training:
            optimizer_RequestedPredictor.zero_grad()
            requested_accumulated_loss = 0
        if not method_predictor_early_stopping.stop_training:
            optimizer_MethodPredictor.zero_grad()
            method_accumulated_loss = 0
        
        for raw_X_train_dialog, train_predictor_Y_dialog in zip(raw_X_train[train_indices], train_predictor_Y[train_indices]):
            
            if not goal_pricerange_predictor_early_stopping.stop_training:
                model_GoalPricerangePredictor.hidden = model_GoalPricerangePredictor.init_hidden()
            if not goal_area_predictor_early_stopping.stop_training:
                model_GoalAreaPredictor.hidden = model_GoalAreaPredictor.init_hidden()
            if not goal_name_predictor_early_stopping.stop_training:
                model_GoalNamePredictor.hidden = model_GoalNamePredictor.init_hidden()
            if not goal_food_predictor_early_stopping.stop_training:
                model_GoalFoodPredictor.hidden = model_GoalFoodPredictor.init_hidden()
            if not requested_predictor_early_stopping.stop_training:
                model_RequestedPredictor.hidden = model_RequestedPredictor.init_hidden()
            if not method_predictor_early_stopping.stop_training:
                model_MethodPredictor.hidden = model_MethodPredictor.init_hidden()
                
            for raw_X_train_turn, train_predictor_Y_turn in zip(raw_X_train_dialog["turns"], train_predictor_Y_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
                
                if not goal_pricerange_predictor_early_stopping.stop_training:
                    goal_pricerange_outputs = model_GoalPricerangePredictor(indices, scores)
                    gold_goal_pricerange = torch.t(torch.tensor([train_predictor_Y_turn["goal_pricerange_predictor"]], dtype = torch.float, device = DEVICE))
                    goal_pricerange_accumulated_loss += PREDICTOR_LOSS_FUNCTION(goal_pricerange_outputs, gold_goal_pricerange)
                if not goal_area_predictor_early_stopping.stop_training:
                    goal_area_outputs = model_GoalAreaPredictor(indices, scores)
                    gold_goal_area = torch.t(torch.tensor([train_predictor_Y_turn["goal_area_predictor"]], dtype = torch.float, device = DEVICE))
                    goal_area_accumulated_loss += PREDICTOR_LOSS_FUNCTION(goal_area_outputs, gold_goal_area)
                if not goal_name_predictor_early_stopping.stop_training:
                    goal_name_outputs = model_GoalNamePredictor(indices, scores)
                    gold_goal_name = torch.t(torch.tensor([train_predictor_Y_turn["goal_name_predictor"]], dtype = torch.float, device = DEVICE))
                    goal_name_accumulated_loss += PREDICTOR_LOSS_FUNCTION(goal_name_outputs, gold_goal_name)
                if not goal_food_predictor_early_stopping.stop_training:
                    goal_food_outputs = model_GoalFoodPredictor(indices, scores)
                    gold_goal_food = torch.t(torch.tensor([train_predictor_Y_turn["goal_food_predictor"]], dtype = torch.float, device = DEVICE))
                    goal_food_accumulated_loss += PREDICTOR_LOSS_FUNCTION(goal_food_outputs, gold_goal_food)
                if not requested_predictor_early_stopping.stop_training:
                    requested_outputs = model_RequestedPredictor(indices, scores)
                    gold_requested = torch.t(torch.tensor([train_predictor_Y_turn["requested_predictor"]], dtype = torch.float, device = DEVICE))
                    requested_accumulated_loss += PREDICTOR_LOSS_FUNCTION(requested_outputs, gold_requested)
                if not method_predictor_early_stopping.stop_training:
                    method_outputs = model_MethodPredictor(indices, scores)
                    gold_method = torch.t(torch.tensor([train_predictor_Y_turn["method_predictor"]], dtype = torch.float, device = DEVICE))
                    method_accumulated_loss += PREDICTOR_LOSS_FUNCTION(method_outputs, gold_method)
        
        if not goal_pricerange_predictor_early_stopping.stop_training:
            goal_pricerange_accumulated_loss.backward()
            optimizer_GoalPricerangePredictor.step()
        if not goal_area_predictor_early_stopping.stop_training:
            goal_area_accumulated_loss.backward()
            optimizer_GoalAreaPredictor.step()
        if not goal_name_predictor_early_stopping.stop_training:
            goal_name_accumulated_loss.backward()
            optimizer_GoalNamePredictor.step()
        if not goal_food_predictor_early_stopping.stop_training:
            goal_food_accumulated_loss.backward()
            optimizer_GoalFoodPredictor.step()
        if not requested_predictor_early_stopping.stop_training:
            requested_accumulated_loss.backward()
            optimizer_RequestedPredictor.step()
        if not method_predictor_early_stopping.stop_training:
            method_accumulated_loss.backward()
            optimizer_MethodPredictor.step()

    dev_losses_dict = get_predictor_loss(model_GoalPricerangePredictor, model_GoalAreaPredictor, model_GoalNamePredictor,
                                         model_GoalFoodPredictor, model_RequestedPredictor, model_MethodPredictor,
                                         raw_X_dev, dev_predictor_Y, DEVICE)
    
    logging.info(dev_losses_dict)

    goal_pricerange_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (-dev_losses_dict["goal_pricerange_loss"]))
    goal_area_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (-dev_losses_dict["goal_area_loss"]))
    goal_name_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (-dev_losses_dict["goal_name_loss"]))
    goal_food_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (-dev_losses_dict["goal_food_loss"]))
    requested_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (-dev_losses_dict["requested_all_loss"]))
    method_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (-dev_losses_dict["method_loss"]))
    
    if goal_pricerange_predictor_early_stopping.wait == 0:
        torch.save(model_GoalPricerangePredictor.state_dict(), "model_PANFMR_incremental_GoalPricerangePredictor.pt")
    if goal_area_predictor_early_stopping.wait == 0:
        torch.save(model_GoalAreaPredictor.state_dict(), "model_PANFMR_incremental_GoalAreaPredictor.pt")
    if goal_name_predictor_early_stopping.wait == 0:
        torch.save(model_GoalNamePredictor.state_dict(), "model_PANFMR_incremental_GoalNamePredictor.pt")
    if goal_food_predictor_early_stopping.wait == 0:
        torch.save(model_GoalFoodPredictor.state_dict(), "model_PANFMR_incremental_GoalFoodPredictor.pt")
    if requested_predictor_early_stopping.wait == 0:
        torch.save(model_RequestedPredictor.state_dict(), "model_PANFMR_incremental_RequestedPredictor.pt")
    if method_predictor_early_stopping.wait == 0:
        torch.save(model_MethodPredictor.state_dict(), "model_PANFMR_incremental_MethodPredictor.pt")
        
    if goal_pricerange_predictor_early_stopping.stop_training:
        model_GoalPricerangePredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerangePredictor.pt"))
    if goal_area_predictor_early_stopping.stop_training:
        model_GoalAreaPredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalAreaPredictor.pt"))
    if goal_name_predictor_early_stopping.stop_training:
        model_GoalNamePredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalNamePredictor.pt"))
    if goal_food_predictor_early_stopping.stop_training:
        model_GoalFoodPredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalFoodPredictor.pt"))
    if requested_predictor_early_stopping.stop_training:
        model_RequestedPredictor.load_state_dict(torch.load("model_PANFMR_incremental_RequestedPredictor.pt"))
    if method_predictor_early_stopping.stop_training:
        model_MethodPredictor.load_state_dict(torch.load("model_PANFMR_incremental_MethodPredictor.pt"))
        
    if goal_pricerange_predictor_early_stopping.stop_training and goal_area_predictor_early_stopping.stop_training and \
        goal_name_predictor_early_stopping.stop_training and goal_food_predictor_early_stopping.stop_training and \
        requested_predictor_early_stopping.stop_training and method_predictor_early_stopping.stop_training:
        break

INFO:root:Epoch	1/50


INFO:root:{'goal_pricerange_loss': 438.41510009765625, 'goal_area_loss': 540.058349609375, 'goal_name_loss': 1.0832583904266357, 'goal_food_loss': 790.7739868164062, 'requested_all_loss': 1080.7735595703125, 'method_loss': 924.8742065429688}
INFO:root:Epoch	2/50


INFO:root:{'goal_pricerange_loss': 421.7687683105469, 'goal_area_loss': 531.9900512695312, 'goal_name_loss': 0.45155227184295654, 'goal_food_loss': 759.9681396484375, 'requested_all_loss': 1066.3040771484375, 'method_loss': 918.0220336914062}
INFO:root:Epoch	3/50


INFO:root:{'goal_pricerange_loss': 415.97833251953125, 'goal_area_loss': 539.110107421875, 'goal_name_loss': 0.26961585879325867, 'goal_food_loss': 767.5510864257812, 'requested_all_loss': 1062.6988525390625, 'method_loss': 892.2531127929688}
INFO:root:Epoch	4/50


INFO:root:{'goal_pricerange_loss': 423.7103271484375, 'goal_area_loss': 543.4491577148438, 'goal_name_loss': 0.18743489682674408, 'goal_food_loss': 802.11767578125, 'requested_all_loss': 1075.654296875, 'method_loss': 892.04638671875}
INFO:root:Epoch	5/50


...

INFO:root:{'goal_pricerange_loss': 415.97833251953125, 'goal_area_loss': 531.9900512695312, 'goal_name_loss': 0.01218899805098772, 'goal_food_loss': 759.9681396484375, 'requested_all_loss': 1062.6988525390625, 'method_loss': 880.8814697265625}
INFO:root:Epoch	47/50


INFO:root:{'goal_pricerange_loss': 415.97833251953125, 'goal_area_loss': 531.9900512695312, 'goal_name_loss': 0.011929563246667385, 'goal_food_loss': 759.9681396484375, 'requested_all_loss': 1062.6988525390625, 'method_loss': 880.8814697265625}
INFO:root:Epoch	48/50


INFO:root:{'goal_pricerange_loss': 415.97833251953125, 'goal_area_loss': 531.9900512695312, 'goal_name_loss': 0.0116738211363554, 'goal_food_loss': 759.9681396484375, 'requested_all_loss': 1062.6988525390625, 'method_loss': 880.8814697265625}
INFO:root:Epoch	49/50


INFO:root:{'goal_pricerange_loss': 415.97833251953125, 'goal_area_loss': 531.9900512695312, 'goal_name_loss': 0.01145597267895937, 'goal_food_loss': 759.9681396484375, 'requested_all_loss': 1062.6988525390625, 'method_loss': 880.8814697265625}
INFO:root:Epoch	50/50


INFO:root:{'goal_pricerange_loss': 415.97833251953125, 'goal_area_loss': 531.9900512695312, 'goal_name_loss': 0.011219613254070282, 'goal_food_loss': 759.9681396484375, 'requested_all_loss': 1062.6988525390625, 'method_loss': 880.8814697265625}


### Load Goal Pricerange Predictor Model

In [37]:
model_GoalPricerangePredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                   embedding_dim = EMBEDDING_DIM,
                                                   altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                   hidden_dim = HIDDEN_DIM,
                                                   output_dim = PREDICTOR_DIM,
                                                   device = DEVICE).to(DEVICE)
model_GoalPricerangePredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerangePredictor.pt"))
model_GoalPricerangePredictor.eval()

iDSTPredictorModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=1, bias=True)
)

### Load Goal Area Predictor Model

In [38]:
model_GoalAreaPredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                             embedding_dim = EMBEDDING_DIM,
                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                             hidden_dim = HIDDEN_DIM,
                                             output_dim = PREDICTOR_DIM,
                                             device = DEVICE).to(DEVICE)
model_GoalAreaPredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalAreaPredictor.pt"))
model_GoalAreaPredictor.eval()

iDSTPredictorModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=1, bias=True)
)

### Load Goal Name Predictor Model

In [39]:
model_GoalNamePredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                             embedding_dim = EMBEDDING_DIM,
                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                             hidden_dim = HIDDEN_DIM,
                                             output_dim = PREDICTOR_DIM,
                                             device = DEVICE).to(DEVICE)
model_GoalNamePredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalNamePredictor.pt"))
model_GoalNamePredictor.eval()

iDSTPredictorModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=1, bias=True)
)

### Load Goal Food Predictor Model

In [40]:
model_GoalFoodPredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                             embedding_dim = EMBEDDING_DIM,
                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                             hidden_dim = HIDDEN_DIM,
                                             output_dim = PREDICTOR_DIM,
                                             device = DEVICE).to(DEVICE)
model_GoalFoodPredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalFoodPredictor.pt"))
model_GoalFoodPredictor.eval()

iDSTPredictorModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=1, bias=True)
)

### Load Requested Predictor Model

In [41]:
model_RequestedPredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                              embedding_dim = EMBEDDING_DIM,
                                              altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                              hidden_dim = HIDDEN_DIM,
                                              output_dim = PREDICTOR_DIM,
                                              device = DEVICE).to(DEVICE)
model_RequestedPredictor.load_state_dict(torch.load("model_PANFMR_incremental_RequestedPredictor.pt"))
model_RequestedPredictor.eval()

iDSTPredictorModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=1, bias=True)
)

### Load Method Predictor Model

In [42]:
model_MethodPredictor = iDSTPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                           embedding_dim = EMBEDDING_DIM,
                                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                           hidden_dim = HIDDEN_DIM,
                                           output_dim = PREDICTOR_DIM,
                                           device = DEVICE).to(DEVICE)
model_MethodPredictor.load_state_dict(torch.load("model_PANFMR_incremental_MethodPredictor.pt"))
model_MethodPredictor.eval()

iDSTPredictorModel(
  (embeddings): Embedding(1129, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=1, bias=True)
)

### Print losses

In [43]:
dev_losses_dict = get_predictor_loss(model_GoalPricerangePredictor, model_GoalAreaPredictor, model_GoalNamePredictor,
                                     model_GoalFoodPredictor, model_RequestedPredictor, model_MethodPredictor,
                                     raw_X_dev, dev_predictor_Y, DEVICE)
pprint(dev_losses_dict)

{'goal_area_loss': 531.9900512695312,
 'goal_food_loss': 759.9681396484375,
 'goal_name_loss': 0.011219613254070282,
 'goal_pricerange_loss': 415.97833251953125,
 'method_loss': 880.8814697265625,
 'requested_all_loss': 1062.6988525390625}


### Plotting

In [45]:
dev_predictor_tracker, dev_predictor_percentage_point, dev_predictor_percentage_points = make_predictor_tracker(model_GoalPricerange, model_GoalArea, model_GoalName,
                                                                                                                model_GoalFood, model_Requested, model_Method,
                                                                                                                model_GoalPricerangePredictor, model_GoalAreaPredictor, model_GoalNamePredictor,
                                                                                                                model_GoalFoodPredictor, model_RequestedPredictor, model_MethodPredictor,
                                                                                                                raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", threshold = 0.30)
print("DEV Predictor percentage point: {}".format(dev_predictor_percentage_point))
dev_predictor_tracker_scores_dict = get_scores(dev_predictor_tracker, dataset = "dstc2_dev", ontology = ontology)
pprint(dev_predictor_tracker_scores_dict)


DEV Predictor percentage point: 0.44
{'goal_area_accuracy': 0.8140162,
 'goal_area_l2': 0.3059282,
 'goal_food_accuracy': 0.6830892,
 'goal_food_l2': 0.4694585,
 'goal_joint_accuracy': 0.5232013,
 'goal_joint_l2': 0.6418665,
 'goal_name_accuracy': 0.9065421,
 'goal_name_l2': 0.1865478,
 'goal_pricerange_accuracy': 0.8165138,
 'goal_pricerange_l2': 0.2682461,
 'method_accuracy': 0.819193,
 'method_l2': 0.2929885,
 'requested_all_accuracy': 0.7818247,
 'requested_all_l2': 0.345153}


In [46]:
plotly_plot_histogram(dev_predictor_percentage_points, "dstc2_dev")

In [47]:
test_predictor_tracker, test_predictor_percentage_point, test_predictor_percentage_points = make_predictor_tracker(model_GoalPricerange, model_GoalArea, model_GoalName,
                                                                                                                   model_GoalFood, model_Requested, model_Method,
                                                                                                                   model_GoalPricerangePredictor, model_GoalAreaPredictor, model_GoalNamePredictor,
                                                                                                                   model_GoalFoodPredictor, model_RequestedPredictor, model_MethodPredictor,
                                                                                                                   raw_X_test, raw_Y_test, "dstc2_test", threshold = 0.30)
print("TEST Predictor percentage point: {}".format(test_predictor_percentage_point))
test_predictor_tracker_scores_dict = get_scores(test_predictor_tracker, dataset = "dstc2_test", ontology = ontology)
pprint(test_predictor_tracker_scores_dict)


TEST Predictor percentage point: 0.4
{'goal_area_accuracy': 0.7953433,
 'goal_area_l2': 0.3311433,
 'goal_food_accuracy': 0.7208207,
 'goal_food_l2': 0.4261224,
 'goal_joint_accuracy': 0.5218289,
 'goal_joint_l2': 0.6595013,
 'goal_name_accuracy': 0.961326,
 'goal_name_l2': 0.0772029,
 'goal_pricerange_accuracy': 0.809034,
 'goal_pricerange_l2': 0.2839782,
 'method_accuracy': 0.8487274,
 'method_l2': 0.2456472,
 'requested_all_accuracy': 0.797187,
 'requested_all_l2': 0.3113484}


In [48]:
plotly_plot_histogram(test_predictor_percentage_points, "dstc2_test")

In [49]:
dev_goal_pricerange_accuracies = []
dev_goal_area_accuracies = []
dev_goal_name_accuracies = []
dev_goal_food_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []
dev_percentages = []

test_goal_pricerange_accuracies = []
test_goal_area_accuracies = []
test_goal_name_accuracies = []
test_goal_food_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []
test_percentages = []

percentages = list(frange(0.1, 1.05, 0.05))
for percentage in tqdm_notebook(percentages, total = len(percentages)):
    
    dev_incremental_tracker, dev_incremental_percentage = make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName,
                                                                       model_GoalFood, model_Requested, model_Method,
                                                                       raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = percentage)
    dev_percentages.append(dev_incremental_percentage)
    dev_scores_dict = get_scores(dev_incremental_tracker, dataset = "dstc2_dev", ontology = ontology)
    dev_goal_pricerange_accuracies.append(dev_scores_dict["goal_pricerange_accuracy"])
    dev_goal_area_accuracies.append(dev_scores_dict["goal_area_accuracy"])
    dev_goal_name_accuracies.append(dev_scores_dict["goal_name_accuracy"])
    dev_goal_food_accuracies.append(dev_scores_dict["goal_food_accuracy"])
    dev_requested_accuracies.append(dev_scores_dict["requested_all_accuracy"])
    dev_method_accuracies.append(dev_scores_dict["method_accuracy"])
    
    test_incremental_tracker, test_incremental_percentage = make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName,
                                                                         model_GoalFood, model_Requested, model_Method,
                                                                         raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = percentage)
    test_percentages.append(test_incremental_percentage)
    test_scores_dict = get_scores(test_incremental_tracker, dataset = "dstc2_test", ontology = ontology)
    test_goal_pricerange_accuracies.append(test_scores_dict["goal_pricerange_accuracy"])
    test_goal_area_accuracies.append(test_scores_dict["goal_area_accuracy"])
    test_goal_name_accuracies.append(test_scores_dict["goal_name_accuracy"])
    test_goal_food_accuracies.append(test_scores_dict["goal_food_accuracy"])
    test_requested_accuracies.append(test_scores_dict["requested_all_accuracy"])
    test_method_accuracies.append(test_scores_dict["method_accuracy"])

In [50]:
plotly_plot_incremental(dev_goal_pricerange_accuracies, dev_goal_area_accuracies, dev_goal_name_accuracies, dev_goal_food_accuracies,
                        dev_requested_accuracies, dev_method_accuracies, dev_percentages, dev_predictor_percentage_point,
                        dev_predictor_tracker_scores_dict["goal_pricerange_accuracy"], dev_predictor_tracker_scores_dict["goal_area_accuracy"], dev_predictor_tracker_scores_dict["goal_name_accuracy"],
                        dev_predictor_tracker_scores_dict["goal_food_accuracy"], dev_predictor_tracker_scores_dict["requested_all_accuracy"], dev_predictor_tracker_scores_dict["method_accuracy"], "dstc2_dev")

In [51]:
plotly_plot_incremental(test_goal_pricerange_accuracies, test_goal_area_accuracies, test_goal_name_accuracies, test_goal_food_accuracies,
                        test_requested_accuracies, test_method_accuracies, test_percentages, test_predictor_percentage_point,
                        test_predictor_tracker_scores_dict["goal_pricerange_accuracy"], test_predictor_tracker_scores_dict["goal_area_accuracy"], test_predictor_tracker_scores_dict["goal_name_accuracy"],
                        test_predictor_tracker_scores_dict["goal_food_accuracy"], test_predictor_tracker_scores_dict["requested_all_accuracy"], test_predictor_tracker_scores_dict["method_accuracy"], "dstc2_test")